In [ ]:
import requests
import sqlite3
import json
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_kode_nama(db_path):
    """Fetch kode and nama pairs from the specified SQLite database."""
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("SELECT kode, nama FROM regions WHERE tingkat = 5")
        all_regions = c.fetchall()
    return all_regions

def fetch_processed_kodes(db_path):
    """Fetch already processed kode from the target database."""
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("SELECT kode FROM election_data")
        processed = c.fetchall()
        processed_kodes = {k[0] for k in processed}
    return processed_kodes

def setup_database(db_path):
    """Set up the SQLite database with the required schema."""
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute('''
            CREATE TABLE IF NOT EXISTS election_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                kode TEXT UNIQUE,
                nama TEXT,
                votes_100025 INTEGER,
                votes_100026 INTEGER,
                votes_100027 INTEGER
            )
        ''')
        conn.commit()

def process_data(kode, nama, db_path):
    """Process data for a given kode and nama by fetching and storing election data."""
    logging.info(f"Starting processing for {kode}")
    json_url = f"https://sirekap-obj-data.kpu.go.id/pemilu/hhcw/ppwp/{kode[:2]}/{kode[:4]}/{kode[:6]}/{kode[:10]}/{kode}.json"
    try:
        response = requests.get(json_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        data = response.json()

        chart = data.get('chart', {}) if data else {}
        votes_100025 = chart.get("100025", 0)
        votes_100026 = chart.get("100026", 0)
        votes_100027 = chart.get("100027", 0)

        with sqlite3.connect(db_path) as conn:
            c = conn.cursor()
            c.execute('''INSERT OR IGNORE INTO election_data (kode, nama, votes_100025, votes_100026, votes_100027)
                         VALUES (?, ?, ?, ?, ?)''', 
                      (kode, nama, votes_100025, votes_100026, votes_100027))
            conn.commit()

        logging.info(f"Finished processing for {kode}")

    except requests.RequestException as e:
        logging.error(f"Request error for {kode}: {e}")
    except KeyError as e:
        logging.error(f"Key error in data for {kode}: {e}")
    except Exception as e:
        logging.error(f"Unexpected error for {kode}: {e}")

def main():
    """Main function to set up database and process all regions with election data."""
    source_db_path = 'data_pemilu_2024_capres.sqlite'
    target_db_path = 'data_TPS_Capres.sqlite'
    
    setup_database(target_db_path)
    processed_kodes = fetch_processed_kodes(target_db_path)
    all_regions = fetch_kode_nama(source_db_path)

    # Filter out already processed regions
    unprocessed_regions = [(kode, nama) for kode, nama in all_regions if kode not in processed_kodes]

    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = [executor.submit(process_data, kode, nama, target_db_path) for kode, nama in unprocessed_regions]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing regions"):
            future.result()  # This will raise any exceptions caught during the execution of the thread
            time.sleep(20)  # Shorter delay to maintain system responsiveness

if __name__ == "__main__":
    main()
